In [1]:
# Text processing designation and description 
from sklearn import neighbors
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data 
X_train = pd.read_csv('/Users/saraskorupa/Documents/rakuten-product-classification/files/X_train_update.csv', index_col='Unnamed: 0')
X_test = pd.read_csv('/Users/saraskorupa/Documents/rakuten-product-classification/files/X_test_update.csv', index_col='Unnamed: 0')
y_train = pd.read_csv('/Users/saraskorupa/Documents/rakuten-product-classification/files/Y_train_CVw08PX.csv', index_col='Unnamed: 0')

# Feature Training Data info
display(X_train.head(10))
X_train.info()

,designation,description,productid,imageid
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786
5,Afrique Contemporaine N° 212 Hiver 2004 - Doss...,NaN,5862738,393356830
6,Christof E: Bildungsprozessen Auf Der Spur,NaN,91920807,907794536
7,Conquérant Sept Cahier Couverture Polypro 240 ...,CONQUERANT CLASSIQUE Cahier 240 x 320 mm seyès...,344240059,999581347
8,Puzzle Scooby-Doo Avec Poster 2x35 Pieces,NaN,4239126071,1325918866
9,Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - ...,Tente pliante V3S5 Pro PVC 500 gr/m² - 3 x 4m5...,3793572222,1245644185


<class 'pandas.core.frame.DataFrame'>
Index: 84916 entries, 0 to 84915
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   designation  84916 non-null  object
 1   description  55116 non-null  object
 2   productid    84916 non-null  int64 
 3   imageid      84916 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 3.2+ MB


In [8]:
'''
# Checking Language in designation
from langdetect import detect

# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'  # In case of any error (e.g., empty or non-standard text)

# Apply the function to your dataframe
X_train['designation_language'] = X_train['designation'].apply(detect_language)

# Check the first few rows to see the results
print(X_train[['designation', 'designation_language']].head())

from langdetect import detect

# Add a language column based on the detected language of the 'designation'
X_train['language'] = X_train['designation'].apply(detect)'''

# Check the distribution of languages
print(X_train['designation_language'].value_counts())

# Check the distribution of languages
print(X_train['designation_language'].value_counts())

# Get the unique languages in the 'designation_language' column
num_languages = X_train['designation_language'].nunique()
print(f"You have {num_languages} unique languages in your data.")


designation_language
fr    51721
en    19620
de     5056
nl     2131
ca     1603
it     1158
ro      739
pt      524
es      503
id      283
no      277
af      189
tl      164
da      147
et      118
sv      105
sw       85
fi       61
so       60
cy       60
pl       54
vi       53
sl       52
hr       49
tr       32
lt       24
hu       16
sk       14
sq        9
cs        5
lv        4
Name: count, dtype: int64
designation_language
fr    51721
en    19620
de     5056
nl     2131
ca     1603
it     1158
ro      739
pt      524
es      503
id      283
no      277
af      189
tl      164
da      147
et      118
sv      105
sw       85
fi       61
so       60
cy       60
pl       54
vi       53
sl       52
hr       49
tr       32
lt       24
hu       16
sk       14
sq        9
cs        5
lv        4
Name: count, dtype: int64
You have 31 unique languages in your data.


In [10]:
# Pre-processing and tokenization
import re

# Function to clean text
def clean_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

# Apply cleaning to the 'designation' column
X_train['cleaned_designation'] = X_train['designation'].apply(clean_text)

# Tokenize the text (split by spaces)
X_train['tokens'] = X_train['cleaned_designation'].apply(lambda x: x.split())



In [21]:
# Stop words removal (language-specific) - ERROR 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


# German Stop words
# Custom stopwords for example language (German in this case)
german_stopwords = set(["der", "die", "das", "und", "in", "zu", "mit", "auf", "für", "von", "sich", "ist"])

def remove_stopwords(tokens, language):
    if language == 'de':  # German
        stop_words = german_stopwords
    else:
        stop_words = stopwords.words(language)
    
    return [word for word in tokens if word not in stop_words]

# Apply the stopword removal process
X_train['tokens_cleaned'] = X_train.apply(
    lambda row: remove_stopwords(row['tokens'], row['designation_language']), axis=1)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saraskorupa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


OSError: No such file or directory: '/Users/saraskorupa/nltk_data/corpora/stopwords/fr'

In [22]:
# Loading spacy packages for lemmentization
'''
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download de_core_news_sm
!python -m spacy download nl_core_news_sm
!python -m spacy download ca_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download ro_core_news_sm
!python -m spacy download pt_core_news_sm
!python -m spacy download pl_core_news_sm
!python -m spacy download es_core_news_sm
'''


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.9 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 6.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 3.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.4 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 6.6 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Lemmatization based on the 10 most common languages
import spacy

# Load spaCy models once to avoid reloading inside the function
language_models = {
    'en': spacy.load('en_core_web_sm'),
    'fr': spacy.load('fr_core_news_sm'),
    'de': spacy.load('de_core_news_sm'),
    'nl': spacy.load('nl_core_news_sm'),
    'ca': spacy.load('ca_core_news_sm'),
    'it': spacy.load('it_core_news_sm'),
    'ro': spacy.load('ro_core_news_sm'),
    'pt': spacy.load('pt_core_news_sm'),
    'pl': spacy.load('pl_core_news_sm'),
    'es': spacy.load('es_core_news_sm')
}

# Function to lemmatize tokens for supported languages
def lemmatize_tokens(tokens, lang):
    # Check if language is supported and load the appropriate spaCy model
    if lang in language_models:
        nlp = language_models[lang]
    else:
        nlp = spacy.load('xx_ent_wiki_sm')  # Default multilingual model
    
    # Apply the model to lemmatize the tokens
    doc = nlp(' '.join(tokens))
    return [token.lemma_ for token in doc]

# Apply lemmatization to the dataset
X_train['lemmatized_tokens'] = X_train.apply(
    lambda row: lemmatize_tokens(row['tokens'], row['designation_language']), axis=1
)

# Check the results for the first few rows
print(X_train[['designation', 'lemmatized_tokens']].head())

In [ ]:
# Text Vectorization 

# TF-IDF (Term Frequency-Inverse Document Frequency):
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_tfidf = tfidf_vectorizer.fit_transform(X_train['lemmatized_tokens'])
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization (Bag of Words)
count_vectorizer = CountVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, token_pattern=None)
X_count = count_vectorizer.fit_transform(X_train['lemmatized_tokens'])




In [6]:
# Product Classifications - most common words

# Flatten the Lemmatized Tokens:
from collections import Counter

    # Flatten the list of all lemmatized tokens
all_tokens = [token for tokens in X_train['lemmatized_tokens'] for token in tokens]

# Count the frequency of each word
word_freq = Counter(all_tokens)

# Display the 10 most common words
most_common_words = word_freq.most_common(10)
print(most_common_words)

# Visualizinh word Cloud 
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()





,designation,description,productid,imageid,designation_language,language
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,de,de
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,fr,fr
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,de,de
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,de,de
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,ca,ca
5,Afrique Contemporaine N° 212 Hiver 2004 - Doss...,NaN,5862738,393356830,fr,fr
6,Christof E: Bildungsprozessen Auf Der Spur,NaN,91920807,907794536,de,de
7,Conquérant Sept Cahier Couverture Polypro 240 ...,CONQUERANT CLASSIQUE Cahier 240 x 320 mm seyès...,344240059,999581347,fr,fr
8,Puzzle Scooby-Doo Avec Poster 2x35 Pieces,NaN,4239126071,1325918866,en,en
9,Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - ...,Tente pliante V3S5 Pro PVC 500 gr/m² - 3 x 4m5...,3793572222,1245644185,fr,fr


In [4]:
# Data Cleaning 
# Filling missing values in description from designation
X_train['description'] = X_train['description'].fillna(X_train['designation'])

# dropping missing values in description 
# X_train.dropna(subset=['description'])
X_train.info()

# drop duplicates
# Check DUBPLICTAES based on all columns
duplicates = X_train[X_train.duplicated()]
# View duplicates
print(duplicates)
# Now drop duplicates if necessary
X_train = X_train.drop_duplicates()



<class 'pandas.core.frame.DataFrame'>
Index: 84916 entries, 0 to 84915
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   designation           84916 non-null  object
 1   description           84916 non-null  object
 2   productid             84916 non-null  int64 
 3   imageid               84916 non-null  int64 
 4   designation_language  84916 non-null  object
 5   description_language  84916 non-null  object
dtypes: int64(2), object(4)
memory usage: 4.5+ MB
Empty DataFrame
Columns: [designation, description, productid, imageid, designation_language, description_language]
Index: []


In [5]:
# Check the distribution of languages
print(X_train['designation_language'].value_counts())

designation_language
fr    51698
en    19613
de     5034
nl     2159
ca     1592
it     1142
ro      745
pt      555
es      522
id      282
no      281
af      185
tl      167
da      142
et      119
sv      103
sw       77
fi       61
cy       59
so       58
sl       56
vi       53
hr       51
pl       48
tr       40
lt       18
sk       16
hu       14
sq       12
cs        7
lv        7
Name: count, dtype: int64


In [14]:
# Text cleaning
import re

# removing special characters, punctuation, and digits
def clean_text(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

X_train['description'] = X_train['description'].apply(clean_text)
X_train['designation'] = X_train['designation'].apply(clean_text)

# Lowercase
X_train['description'] = X_train['description'].str.lower()
X_train['designation'] = X_train['designation'].str.lower()


In [30]:
# Removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

X_train['description'] = X_train['description'].apply(remove_stopwords)
X_train['designation'] = X_train['designation'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saraskorupa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
from nltk.tokenize import PunktSentenceTokenizer
tokenizer = PunktSentenceTokenizer()
tokenizer.tokenize(X_train['description'][0])

['Olivia: Personalisiertes Notizbuch / 150 Seiten / Punktraster / Ca Din A5 / Rosen-Design']

In [29]:
# Tokenization of description
import spacy

# Load the French language model
nlp_fr = spacy.load('fr_core_news_sm')

# Tokenize the first 10 rows of the description variable
X_train['description_tokenized'] = X_train['description'].head(10).apply(lambda x: [token.text for token in nlp_fr(x)])

# Display the result
print(X_train[['description', 'description_tokenized']].head(10))



                                         description  \
0  Olivia: Personalisiertes Notizbuch / 150 Seite...   
1  Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...   
2  PILOT STYLE Touch Pen de marque Speedlink est ...   
3  Peluche Donald - Europe - Disneyland 2000 (Mar...   
4  Luc des id&eacute;es de grandeur. Il veut orga...   
5  Afrique Contemporaine N° 212 Hiver 2004 - Doss...   
6         Christof E: Bildungsprozessen Auf Der Spur   
7  CONQUERANT CLASSIQUE Cahier 240 x 320 mm seyès...   
8          Puzzle Scooby-Doo Avec Poster 2x35 Pieces   
9  Tente pliante V3S5 Pro PVC 500 gr/m² - 3 x 4m5...   

                               description_tokenized  
0  [Olivia, :, Personalisiertes, Notizbuch, /, 15...  
1  [Journal, Des, Arts, (, Le, ), N, °, 133, Du, ...  
2  [PILOT, STYLE, Touch, Pen, de, marque, Speedli...  
3  [Peluche, Donald, -, Europe, -, Disneyland, 20...  
4  [Luc, des, id&eacute;es, de, grandeur, ., Il, ...  
5  [Afrique, Contemporaine, N, °, 212, Hiver, 200... 

In [33]:
# Lemmatization
import spacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Function to lemmatize text
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

# Apply lemmatization to the 'description' and 'designation' columns
X_train['description'] = X_train['description'].apply(lemmatize_text)
X_train['designation'] = X_train['designation'].apply(lemmatize_text)

KeyboardInterrupt: 

In [20]:
import pandas as pd

# Function to process text in batches
def process_in_batches(df, batch_size=1000):
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))
        batch = df.iloc[start:end]
        # Apply lemmatization here to the batch
        batch['designation'] = batch['designation'].apply(lemmatize_text)
        batch['description'] = batch['description'].apply(lemmatize_text)
        # You can save or process the batch results here
        yield batch

# Example usage:
for batch in process_in_batches(X_train):
    # Process each batch (e.g., saving or analyzing results)
    pass



/var/folders/z4/tms7jyg136v2x9gmjn8y6pd40000gn/T/ipykernel_85596/3170184494.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['designation'] = batch['designation'].apply(lemmatize_text)
/var/folders/z4/tms7jyg136v2x9gmjn8y6pd40000gn/T/ipykernel_85596/3170184494.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['description'] = batch['description'].apply(lemmatize_text)


In [ ]:
# language detection - language translate 
from googletrans import Translator

translator = Translator()

def translate_text(text, target_language='en'):
    try:
        translated = translator.translate(text, dest=target_language)
        return translated.text
    except Exception as e:
        return text  # Return the original text if there's an error

# Apply translation to 'description' and 'designation' columns
X_train['description'] = X_train['description'].apply(lambda x: translate_text(x, 'en') if pd.notnull(x) else x)
X_train['designation'] = X_train['designation'].apply(lambda x: translate_text(x, 'en') if pd.notnull(x) else x)
